# Estudo Algoritmo de Classificação Descision Tree
- O objetivo deste notebook é o estudo do algoritmo de classificação Descision Tree do scikit-learn.
- Vamos utilizar nesse estudo o data set 'census.csv'.
- Este dataset contém 15 atributos e 32.561 instâncias (linhas). 

  <h2>Principais Vantagens x Desvantagens </h2>
- <h3>Vantagens </h3>
- Não precisa normalização ou padronização
- Simplicidade de interpretação
- Rápido para classificar novos registros
-  <h3>Desvantagens </h3>
- Geração de árvores muito complexas
- Pequenas mudanças nos dados pode mudar a árvore (poda pode ajudar)
- Problema NP-completo para construir a árvore

In [1]:
# Importando as bibliotecas iniciais
import pandas as pd
import numpy as np

In [2]:
# Importa o dataset
base = pd.read_csv('./census.csv')

In [3]:
# Verificando o shape do dataset
base.shape

(32561, 15)

In [4]:
# Visualiza as primeira linhas
base.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
# Verificando os atributos núméricos.
base.describe()

,age,final-weight,education-num,capital-gain,capital-loos,hour-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


# Pré-processamento - Tratamento dos dados

In [6]:
# Seleciona os atributos previsores
previsores = base.iloc[:, 0:14].values
previsores

array([[39, ' State-gov', 77516, ..., 0, 40, ' United-States'],
       [50, ' Self-emp-not-inc', 83311, ..., 0, 13, ' United-States'],
       [38, ' Private', 215646, ..., 0, 40, ' United-States'],
       ...,
       [58, ' Private', 151910, ..., 0, 40, ' United-States'],
       [22, ' Private', 201490, ..., 0, 20, ' United-States'],
       [52, ' Self-emp-inc', 287927, ..., 0, 40, ' United-States']],
      dtype=object)

In [7]:
# Seleciona a classe alvo (target)
classe = base.iloc[:,14]
#classe

In [8]:
# Realiza a codificação dos atributos categóricos. Transforma as categorias em códigos.
from sklearn.preprocessing import LabelEncoder

label_encoder_previsores = LabelEncoder()
previsores[:,1] = label_encoder_previsores.fit_transform(previsores[:,1])
previsores[:,3] = label_encoder_previsores.fit_transform(previsores[:,3])
previsores[:,5] = label_encoder_previsores.fit_transform(previsores[:,5])
previsores[:,6] = label_encoder_previsores.fit_transform(previsores[:,6])
previsores[:,7] = label_encoder_previsores.fit_transform(previsores[:,7])
previsores[:,8] = label_encoder_previsores.fit_transform(previsores[:,8])
previsores[:,9] = label_encoder_previsores.fit_transform(previsores[:,9])
previsores[:,13] = label_encoder_previsores.fit_transform(previsores[:,13])
previsores

array([[39, 7, 77516, ..., 0, 40, 39],
       [50, 6, 83311, ..., 0, 13, 39],
       [38, 4, 215646, ..., 0, 40, 39],
       ...,
       [58, 4, 151910, ..., 0, 40, 39],
       [22, 4, 201490, ..., 0, 20, 39],
       [52, 5, 287927, ..., 0, 40, 39]], dtype=object)

In [9]:
# Realiza a binarização dos valores inteiros. Técnica chamada: Dummy variables.
# Isto é importante para que o algoritmo não considere um valor maior mais importante que um valor menor.
# Pelos testes verificamos que se usarmos somente o OneHotEncoder sem realizar o escalonamento temos uma pequena melhora na precisão do Modelo. A acurácia fica em torno de 84.61%. Por isso, vamos deixar ele habilitado.

from sklearn.preprocessing import OneHotEncoder

oneHotEnconder = OneHotEncoder(categories='auto')
previsores = oneHotEnconder.fit_transform(previsores).toarray()
previsores

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [10]:
# StandardScaler para colocar todos os valores na mesma escala (escalonamento).
# Teste do escalonamente de todos atributos, também não tivemos melhoria no modelo. Tivemos o mesma Acurácia de 81.08%. Por isso, vamos deixar ele dasabilitado.

#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#previsores = scaler.fit_transform(previsores)

In [11]:
# StandardScaler para colocar todos os valores na mesma escala (escalonamento).
# Teste do escalonamente somente dos atributos inteiros, também não tivemos melhoria no modelo. Tivemos o mesma Acurácia de 81.07%. Por isso, vamos deixar ele dasabilitado também . 

#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()

#previsores[:,0] =  scaler.fit_transform(previsores[:,0].reshape(-1,1)).ravel()
#previsores[:,2] =  scaler.fit_transform(previsores[:,2].reshape(-1,1)).ravel()
#previsores[:,4] =  scaler.fit_transform(previsores[:,4].reshape(-1,1)).ravel()
#previsores[:,10] =  scaler.fit_transform(previsores[:,10].reshape(-1,1)).ravel()
#previsores[:,11] =  scaler.fit_transform(previsores[:,11].reshape(-1,1)).ravel()
#previsores[:,12] =  scaler.fit_transform(previsores[:,12].reshape(-1,1)).ravel()


In [12]:
# Separa os dados pata treinamento e teste
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe , test_size = 0.25, random_state = 0)


# Criando o Modelo de Previsão

In [13]:
# Cria o modelo de previsão DecisioncTree Classifier

from sklearn.tree import DecisionTreeClassifier
# A função para medir a qualidade de seleção dos nós sera a entropia 'entropy'.
classificador = DecisionTreeClassifier(criterion='entropy', random_state=0)
classificador.fit(previsores_treinamento, classe_treinamento)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [14]:
# Realizando o testes do modelo
classe_prevista_teste = classificador.predict(previsores_teste)

# Verificando a performance do Modelo

In [15]:
# Analisando a performance do Modelo
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
precisao_modelo = accuracy_score(classe_teste, classe_prevista_teste)
print('Acurácia do Modelo: %.2f%% ' % (precisao_modelo*100))


Acurácia do Modelo: 84.61% 


In [16]:
# Relatório de Classificação
from sklearn.metrics import classification_report

#precision -  daqueles que classifiquei como corretos, quantos efetivamente estavam corretos? 
#recall    -  quando realmente é da classe XPTO, o quão frequente você classifica como XPTO?
#f1-score  -  balanço entre a precisão e o recall. Combina precisão e recall em um número único que indique a qualidade geral do seu modelo (quanto maior melhor o modelo).
#support   -  número de ocorrência de cada classe.

relatorio_classificacao = classification_report(classe_teste, classe_prevista_teste ,digits=4)

# Imprimindo o relatório
print('\tRelatório de Classificação do modelo')
print(relatorio_classificacao)

	Relatório de Classificação do modelo
              precision    recall  f1-score   support

       <=50K     0.8822    0.9193    0.9004      6159
        >50K     0.7115    0.6186    0.6618      1982

    accuracy                         0.8461      8141
   macro avg     0.7969    0.7689    0.7811      8141
weighted avg     0.8407    0.8461    0.8423      8141



In [17]:
# Visualizando a Matriz de confusão (confusion matrix)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(classe_teste, classe_prevista_teste) 

df_cm = pd.DataFrame(cm)

print('\n')
print('Obs.: Nas linhas temos as Classes reais e nas colunas as Classes previstas.\n')
print('Matriz de Confusão')
df_cm



Obs.: Nas linhas temos as Classes reais e nas colunas as Classes previstas.

Matriz de Confusão


,0,1
0,5662,497
1,756,1226


In [18]:
# Comparando a Classe Prevista com a Classe Real

df = pd.DataFrame( )
df['Classe_Prevista'] = classe_prevista_teste
df['Classe_Real'] = classe_teste.values
df['Comparacao'] = df['Classe_Prevista'] == df['Classe_Real']

df_comparacao = df.groupby(by='Comparacao').count()

print('De %d classes o modelo ACERTOU %d classes e ERROU %d classes.\n ' % (df_comparacao['Classe_Prevista'].sum(), df_comparacao.iloc[[1], [0]].values, df_comparacao.iloc[[0], [0]].values))

df_comparacao

De 8141 classes o modelo ACERTOU 6888 classes e ERROU 1253 classes.
 


,Classe_Prevista,Classe_Real
Comparacao,,
False,1253,1253
True,6888,6888


# Resultados do Algoritmo Decision Tree

<h3>Testes realizados:</h3>

- Precisão: 0.8105 - Decision Tree com: LabelEncoder
- Precisão: 0.8461 - Decision Tree com: LabelEncoder + OneHotEncoder
- Precisão: 0.8461 - Decision Tree com: LabelEncoder + OneHotEncoder + StandardScaler (Total)
- Precisão: 0.8461 - Decision Tree com: LabelEncoder + OneHotEncoder + StandardScaler (Parcial)
- Precisão: 0.8108 - Decision Tree com: LabelEncoder + StandardScale (Total)
- Precisão: 0.8107 - Decision Tree com: LabelEncoder + StandardScale (Parcial)


# Resultados do Algoritmo Naive Bayes


<h3>Testes realizados:</h3>

- Precisão: 0.7947 - Naive Bayes com: LabelEncoder
- Precisão: 0.5041 - Naive Bayes com: LabelEncoder + OneHotEncoder
- Precisão: 0.4922 - Naive Bayes com: LabelEncoder + OneHotEncoder + StandardScaler (Total)
- Precisão: 0.5041 - Naive Bayes com: LabelEncoder + OneHotEncoder + StandardScaler (Parcial)
- Precisão: 0.8046 - Naive Bayes com: LabelEncoder + StandardScale (Total)
- Precisão: 0.8046 - Naive Bayes com: LabelEncoder + StandardScale (Parcial)

<h3>Link do Notebook dos testes do  Algoritmo Naive Bayes</h3>
<a href="https://github.com/charlessodre/Charles_Estudo_Algoritmo_Naive_Bayes_Census">Resultados do Algoritmo Naive Bayes</a>

# Conclusão:

- Podemos observar nos resultados acima que o 'Algoritmo Decision Tree' obteve um melhor resultado em relação ao 'Algoritmo Naive Bayes' para o dataset 'census.csv' quando utilizado os mesmos pré-processamentos .